# Redes Sociais - APS 2

### Alunos: Arthur Barreto, Enricco Gemha e Felipe Catapano


Uma rede de artistas do Spotify que se conectam através de participar em determinada música, que foi topo das paradas em alguma semana no mundo, durante o período de 28/09/2013 a 09/10/2022. Os vértices representam artistas do Spotify e uma aresta não-direcionada indica uma música feita em parceria por dois artistas.

_Nota: os grafos são não-dirigidos, conforme informado no agregador de databases [Kaggle](https://www.kaggle.com/datasets/jfreyberg/spotify-artist-feature-collaboration-network), do qual foi extraído a base utilizada aqui._

## Pré-requisitos

In [ ]:
import graph_tool_extras as gte
import netpixi
from graph_tool import draw
import distribution as dst

In [ ]:
PATH = 'edges.csv'
PATH_NOS = 'nodes.csv'

## Análise dos dados importados

O arquivo relevante à análise inicial é `edges.csv`, que representa uma lista de valores separados por _vírgulas_, com uma aresta por linha. O significado de cada coluna do arquivo é:

- Primeira coluna: ID do nó de participação em música
- Segunda coluna: ID do nó de participação em música

In [ ]:
with open(PATH) as file:

    # cria index de contagem para o loop abaixo.
    i = 0
    
    # ignora o cabeçalho.
    next(file)

    # Para não sobrecarregar este notebook
    # vamos espiar somente as 5 primeiras linhas.
    for line in file:

        # Transforma a linha em uma lista de partes,
        # considerando a vírgula como separador.
        parts = line.split(',')
        no1 = parts[0]
        no2 = parts[1].replace('\n', '')

        print(f'{no1} - {no2}')
        
        # Para não sobrecarregar este notebook, vamos usar um contador
        # e um break para imprimir apenas as cinco primeiras linhas.
        i += 1
        if i == 5:
            break

## Criação do grafo

Utilizaremos a biblioteca [graph-tool](https://graph-tool.skewed.de/) somente para criação e visualização básica dos grafos, sem suporte de nenhum método ou função que não seja essencial.

In [ ]:
g = gte.Graph(directed=False) # pois o grafo não é direcionado, como informado acima.

Antes, vamos definir duas funções auxiliares para facilitar a adição de novos nós e arestas, respectivamente.

In [ ]:
def get_or_add_vertex(g, id):
    u = g.vertex_by_id(id)
    if u is None:
        u = g.add_vertex_by_id(id)
    return u

def get_or_add_edge(g, id1, id2):
    e = g.edge_by_ids(id1, id2)
    if e is None:
        e = g.add_edge_by_ids(id1, id2)
    return e

Depois de criar o novo grafo, vamos armazenar os valores de `edges.csv` nele, o transformando em uma rede.

In [ ]:
with open(PATH) as file:

    # Cria index de contagem de linhas lidas.
    i = 1
    
    # Ignora o cabeçalho.
    next(file)

    # Itera linha a linha do arquivo `out.linux`
    for line in file:

        # Transforma a linha em uma lista de partes,
        # considerando a vírgula como separador.
        parts = line.split(',')

        # Define os IDs de origem e destino.
        no1 = parts[0]
        no2 = parts[1].replace('\n', '')

        # Adiciona os vértices.
        get_or_add_vertex(g, no1)
        get_or_add_vertex(g, no2)
    
        # Adiciona a aresta correspondente a esta linha.
        get_or_add_edge(g, no1, no2)
        
        # Incrementa o contador de linhas lidas.
        i += 1

# Imprime a quantidade de linhas lidas.
print(f'Foram lidas {i} linhas.') 

A seguir, devemos chamar `draw.sfdp_layout`, passando a rede, para rodar um algoritmo de posicionamento chamado SFDP [[1](#sfdp)].

Esse algoritmo usa uma ideia conhecida como [force-directed graph drawing](https://en.wikipedia.org/wiki/Force-directed_graph_drawing) para posicionar os vértices de forma a evidenciar agrupamentos.

In [ ]:
layout = draw.sfdp_layout(g)

In [ ]:
gte.move(g, layout)

## Armazenamento da rede

Para garantir a segurança da informação processada, devemos guardá-la em um arquivo na mesma pasta deste notebook.

In [ ]:
g = gte.clean(g)
gte.save(g, 'spotify.net.gz')

## Visualização da rede

O próximo passo é a renderização da rede.

In [ ]:
r = netpixi.render('spotify.net.gz', infinite=True)

Por fim, devemos ajustar a visualização da renderização.

In [ ]:
r.vertex_default(size=4, bwidth=1)

In [ ]:
r.edge_default(width=1)

## Estatísticas

Agora, devemos ver o tamanho da rede, ou seja, a quantidade de _nodes_ (nós) e _edges_ (arestas).

In [ ]:
n = len(g.get_vertices())
m = len(g.get_edges())
print(f"Número de nós: {n}")
print(f"Número de arestas: {m}")

Afim de compreender melhor a rede, devemos obter os valores de **densidade** e **transitividade** da rede.

A densidade da rede é a razão entre número de pares conectados, dentre todos os pares.

Por outro lado, a transitividade aponta a razão de trios de nós que estão conectados, dentre todos os trios, na forma de triângulos.

*obs: calculamos "na mão" o valor da densidade porque o valor é truncado pelo método `density()` do graph-tool.*

In [ ]:
m_max = n*(n-1)/2 # fórmula de m_max para grafos não dirigidos
densidade_g = m/m_max # fórmula da densidade de um grafo
print(f"Densidade da rede: {(densidade_g*100):e}%") # valor em notação científica
print(f"Transitividade da rede: {g.transitivity()}%")

Buscamos obter em seguida uma `Series` (_type_ do Pandas), em que cada vértice possui um valor para o número de vizinhos que possui. 

In [ ]:
degrees = g.get_total_degrees() # obtem a quantidade de vizinhos dos vértices

In [ ]:
degrees.describe() # por ser uma série do Pandas, utilizamos o método describe()

In [ ]:
import numpy as np
degrees.hist(bins = np.arange(1,11,1));

In [ ]:
dst.not_normal(degrees)

In [ ]:
dst.more_powerlaw_than_lognormal(degrees)

In [ ]:
dst.more_powerlaw_than_exponential(degrees)

### Estratificando a amostra pelo número de nós

Devido ao fato do método distance ser $O(n^2)$ é necessário estratificar para que o algoritmo consiga rodar (já que não há memória RAM suficiente disponível).

In [ ]:
dict_nos = {}

with open(PATH_NOS) as file:

    # cria index de contagem para o loop abaixo.
    i = 0
    
    # ignora o cabeçalho.
    next(file)

    # Para não sobrecarregar este notebook
    # vamos espiar somente as 5 primeiras linhas.
    for line in file:

        # Transforma a linha em uma lista de partes,
        # considerando a vírgula como separador.
        parts = line.split(',')

        id = parts[0]

        if id not in dict_nos:
            dict_nos[id] = i
            i += 1

print(f'Contamos {i} nós!')
print(f'O dicionário tem {len(list(dict_nos.keys()))} chaves')

Visualizando os valores do dicionário:

In [ ]:
list(dict_nos.keys())[:5]

In [ ]:
list(dict_nos.values())[:5]

A abordagem será sortear indices, que variam de 1 até a quantidade de nós `i`. Após isso, basta verificar os índices sorteados que estão na lista de valores do dicionário criado.

Selecionamos, portanto, 15.000 nós aleatórios, para que não estouremos a quantidade limite em memória RAM.

In [ ]:
import numpy as np
from numpy.random import choice

lisIdx = np.arange(1, i + 1, 1)
lisProb = np.ones(shape=len(lisIdx), dtype=lisIdx.dtype)
lisProb = np.divide(lisProb, len(lisProb))
idxSort = choice(lisIdx, int(15e3), p = lisProb, replace = False)
idxSort.sort()
idxSort[:5]

Para facilitar a verificação dos nos considerados, basta inverter o dicionário, desta forma, as chaves serão os índices e os valores os nós em si.

In [ ]:
idx_no = {v: k for k, v in dict_nos.items()}

In [ ]:
list(idx_no.keys())[:5], list(idx_no.values())[:5]

In [ ]:
noStrat = {idx_no[idx]: idx for idx in idxSort}

In [ ]:
list(noStrat.values())[:5]

### Criando um novo grafo `g_strat`

Realizamos abaixo o mesmo procedimento que foi aplicado ao grafo principal `g`, com a diferença de que agora estaremos utilizando somente os nós estratificados, salvando em `g_strat`.

In [ ]:
g_strat = gte.Graph(directed=False) # pois o grafo não é direcionado, como informado acima.

In [ ]:
with open(PATH) as file:

    # Cria index de contagem de linhas lidas.
    i = 1
    j = 0
    
    # Ignora o cabeçalho.
    next(file)

    # Itera linha a linha do arquivo
    for line in file:

        # Transforma a linha em uma lista de partes,
        # considerando a vírgula como separador.
        parts = line.split(',')

        # Define os IDs de origem e destino.
        no1 = parts[0]
        no2 = parts[1].replace('\n', '')

        if (no1 and no2) in noStrat:
            # Adiciona os vértices somente para os nós estratificados.
            get_or_add_vertex(g_strat, no1)
            get_or_add_vertex(g_strat, no2)
    
            # Adiciona a aresta correspondente a esta linha.
            get_or_add_edge(g_strat, no1, no2)

            j += 1
        
        # Incrementa o contador de linhas lidas.
        i += 1

# Imprime a quantidade de linhas lidas.
print(f'Foram lidas {i} linhas.') 
print(f'Foram considerados {j} arestas')

In [ ]:
layout_strat = draw.sfdp_layout(g_strat)

In [ ]:
gte.move(g_strat, layout_strat)

In [ ]:
g_strat = gte.clean(g_strat)
gte.save(g_strat, 'spotify_strat.net.gz')

### Obtendo a Distribuição de Distância

In [ ]:
distances = g_strat.get_distances()

In [ ]:
distances.describe()

In [ ]:
distances.hist();